<a href="https://colab.research.google.com/github/Ryuga-17/Car-Price-Predictor-XgBOOST/blob/main/XgBoost_Car_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df=pd.read_csv('/content/cars_data.csv')

In [ ]:
df.head()

,Make,Model,Type,Origin,DriveTrain,MSRP,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,"$36,945",3.5,6.0,265,17,23,4451,106,189
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820",2.0,4.0,200,24,31,2778,101,172
2,Acura,TSX 4dr,Sedan,Asia,Front,"$26,990",2.4,4.0,200,22,29,3230,105,183
3,Acura,TL 4dr,Sedan,Asia,Front,"$33,195",3.2,6.0,270,20,28,3575,108,186
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,"$43,755",3.5,6.0,225,18,24,3880,115,197


In [ ]:
df.isnull().sum()

,0
Make,0
Model,0
Type,0
Origin,0
DriveTrain,0
MSRP,0
EngineSize,0
Cylinders,2
Horsepower,0
MPG_City,0


In [ ]:
df.shape

(428, 14)

In [ ]:
len(df['Model'].unique())
#clearly its wise to not consider this feature

425

In [ ]:
df['Type'].unique()

array(['SUV', 'Sedan', 'Sports', 'Wagon', 'Truck', 'Hybrid'], dtype=object)

In [ ]:
df['DriveTrain'].unique()

array(['All', 'Front', 'Rear'], dtype=object)

In [ ]:
mean=np.mean(df['Cylinders'])

df['Cylinders'].fillna(mean,inplace=True)

<ipython-input-10-425ca97a5f1f>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Cylinders'].fillna(mean,inplace=True)


In [ ]:
df.isnull().sum()

,0
Make,0
Model,0
Type,0
Origin,0
DriveTrain,0
MSRP,0
EngineSize,0
Cylinders,0
Horsepower,0
MPG_City,0


In [ ]:
df['Origin'].unique()

array(['Asia', 'Europe', 'USA'], dtype=object)

In [ ]:
df['DriveTrain'].unique()

array(['All', 'Front', 'Rear'], dtype=object)

In [ ]:
df['MSRP']=df['MSRP'].str.replace('$','')
df['MSRP']=df['MSRP'].str.replace(',','').astype(float)

In [ ]:
y=df['MSRP']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
ohe=OneHotEncoder(drop='first')


In [ ]:
from sklearn.linear_model import LinearRegression
from  sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score


In [ ]:
hf=df.drop(['Model','MSRP'],axis=1)

In [ ]:
hf.head()

,Make,Type,Origin,DriveTrain,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,SUV,Asia,All,3.5,6.0,265,17,23,4451,106,189
1,Acura,Sedan,Asia,Front,2.0,4.0,200,24,31,2778,101,172
2,Acura,Sedan,Asia,Front,2.4,4.0,200,22,29,3230,105,183
3,Acura,Sedan,Asia,Front,3.2,6.0,270,20,28,3575,108,186
4,Acura,Sedan,Asia,Front,3.5,6.0,225,18,24,3880,115,197


In [ ]:
hf['Type'].unique()

array(['SUV', 'Sedan', 'Sports', 'Wagon', 'Truck', 'Hybrid'], dtype=object)

In [ ]:
hf['Origin'].unique()

array(['Asia', 'Europe', 'USA'], dtype=object)

In [ ]:
len(hf['DriveTrain'].unique())

3

In [ ]:
len(hf['Make'].unique())

38

In [ ]:
hf.shape

(428, 12)

In [ ]:
pf=ohe.fit_transform(hf.iloc[:,0:4]).toarray()

In [ ]:
pf=pd.DataFrame(pf)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(hf.iloc[:,4:])
gh=scaler.transform(hf.iloc[:,4:])
gh=pd.DataFrame(gh)


In [ ]:
q=pd.concat([gh,pf],axis=1)
q.dtypes
q = q.to_numpy()


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
q_train,q_test,y_train,y_test=train_test_split(q,y,test_size=0.23,random_state=42)

In [ ]:
rf=RandomForestRegressor()
rf.fit(q_train,y_train)
y_pred=rf.predict(q_test)
rf.score(q_test,y_test)

0.9024958192729693

In [ ]:
lr=LinearRegression()
lr.fit(q_train,y_train)
y_pred=lr.predict(q_test)
lr.score(q_test,y_test)

0.8341683577173935

In [ ]:
from xgboost import XGBRegressor
xg=XGBRegressor()
xg.fit(q_train,y_train)
y_pred=xg.predict(q_test)
xg.score(q_test,y_test)

0.8937042093770087

In [ ]:
p=df['MSRP']

In [ ]:
hf.head()

,Make,Type,Origin,DriveTrain,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,SUV,Asia,All,3.5,6.0,265,17,23,4451,106,189
1,Acura,Sedan,Asia,Front,2.0,4.0,200,24,31,2778,101,172
2,Acura,Sedan,Asia,Front,2.4,4.0,200,22,29,3230,105,183
3,Acura,Sedan,Asia,Front,3.2,6.0,270,20,28,3575,108,186
4,Acura,Sedan,Asia,Front,3.5,6.0,225,18,24,3880,115,197


In [ ]:
x_train,x_test,p_train,p_test=train_test_split(hf,p,test_size=0.23,random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

tr1=ColumnTransformer([('scaler',MinMaxScaler(),[4,5,6,7,8,9,10,11]),('ohe',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[0,1,3,2])],remainder='passthrough')

xgpipe=Pipeline([('tr1',tr1),('xg2',XGBRegressor())])



In [ ]:
xgpipe.fit(x_train,y_train)

Pipeline(steps=[('tr1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  [4, 5, 6, 7, 8, 9, 10, 11]),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0, 1, 3, 2])])),
                ('xg2',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              ea...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
xgpipe.score(x_test,y_test)

0.8962256333849086

In [ ]:
import pickle
with open ('xgbpipe.pkl','wb') as f:
  pickle.dump(xgpipe,f)

In [ ]:
params={"learning_rate":[0.05, 0.10,0.15,0.20,0.25,0.30],
        "max_depth":[3,4,5,6,8,10,12,15],
        "min_child_weight":[1,3,5,7],
        "gamma":[0.0,0.1,0.2,0.3,0.4],
        "colsample_bytree":[0.3,0.4,0.5,0.7]}


In [ ]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
r_s=RandomizedSearchCV(xg,param_distributions=params,n_iter=10,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
r_s.fit(X_train,y_train)


NameError: name 'X_train' is not defined

In [ ]:
r_s.best_params_

In [ ]:
r_s.best_estimator_

In [ ]:
xgb2=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.4, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None, max_child_weight=1,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=None, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None)

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(xgb2,X_train,y_train,cv=10)

In [ ]:
score

In [ ]:
with open('xgb2.pkl','wb') as f:
  pickle.dump(xgb2,f)